In [1]:
import cv2 as cv
import cv2.aruco as aruco
import numpy as np
import os
clear = lambda: os.system('cls')
clear()
from IPython.display import clear_output

ModuleNotFoundError: No module named 'cv2'

In [2]:
help(cv.aruco)

NameError: name 'cv' is not defined

In [2]:
aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
print(aruco_dict.markerSize)

6


In [3]:
img = aruco.drawMarker(aruco_dict, id = 1, sidePixels = 700)
cv.imshow('Marker preview', img)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('Marker_1.jpg', img)

NameError: name 'aruco' is not defined

In [6]:
help(aruco.Board_create)

Help on built-in function Board_create:

Board_create(...)
    Board_create(objPoints, dictionary, ids) -> retval
    .   * @brief Provide way to create Board by passing nessesary data. Specially needed in Python.
    .   *
    .   * @param objPoints array of object points of all the marker corners in the board
    .   * @param dictionary the dictionary of markers employed for this board
    .   * @param ids vector of the identifiers of the markers in the board
    .   *



In [2]:
def capOneFrame(camNum):
    cap = cv.VideoCapture(camNum)
    while True:
        grabbed, img = cap.read()
        cv.imshow('CAPTURE ONE FRAME FUNC, click q to grab frame', img)
        if cv.waitKey(5) & 0xFF == ord('q'):
            break;
    return img if grabbed else -1

In [17]:
def show(img, windName='', destroyAll=False):
    cv.imshow(windName, img);cv.waitKey(0)
    if destroyAll: cv.destroyAllWindows()

In [25]:
cap = cv.VideoCapture(1)
count = 0
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
    parameters = aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    
    frame_with_markers = aruco.drawDetectedMarkers(frame, corners)
    
    print(corners)
    cv.imshow('frame_with_markers', frame)
    
    ## printing only 10, and other less importand things
    if count==10:
        clear_output() #clear()
        count=0;
    count+=1
    if cv.waitKey(5) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()


[]
[]
[]


In [22]:
def detectMarkers():
    frame = cv.imread('detectMarkers_1.png')# capOneFrame(1)
    show(frame, 'Input picture')
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
    parameters = aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = \
    aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
    frame_with_markers = aruco.drawDetectedMarkers(frame, corners)
    print(corners)
    print('Order is: RB, LB, LT, RT,zgodnie ze wzkazówkami zegara od punktu początkowego')
    show(frame, 'Detected Marker', destroyAll=True)
detectMarkers()

[array([[[348., 284.],
        [214., 281.],
        [214., 148.],
        [350., 146.]]], dtype=float32)]
Order is: RB, LB, LT, RT,zgodnie ze wzkazówkami zegara od punktu początkowego


In [9]:
n_frame = frame.copy()
print(f"Wymiary klatki: {n_frame.shape}")
canny = cv.Canny(n_frame, 100, 200)
img = cv.copyMakeBorder(canny, 25, 25, 25, 25,
                            cv.BORDER_CONSTANT, value=255)
print(f"edges shape: {img.shape}")
im2, contours, hierarchy  = cv.findContours(img, cv.RETR_CCOMP, cv.CHAIN_APPROX_SIMPLE)
contours.sort(key=cv.contourArea, reverse=True)
withContours = np.zeros(im2.shape, dtype="uint8")
cv.drawContours(withContours, contours, -1, (255,0,0), thickness=1)
cv.imshow('Preview', withContours)
cv.imshow('Contours', im2)
cv.waitKey(0)
cv.destroyAllWindows()

NameError: name 'frame' is not defined

In [98]:
# help(aruco.drawDetectedMarkers)
print(len(contours))
#long = max(contours, key=len)
contours.sort(key=len)

189


In [79]:
help(cv.drawContours)

Help on built-in function drawContours:

drawContours(...)
    drawContours(image, contours, contourIdx, color[, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) -> image
    .   @brief Draws contours outlines or filled contours.
    .   
    .   The function draws contour outlines in the image if \f$\texttt{thickness} \ge 0\f$ or fills the area
    .   bounded by the contours if \f$\texttt{thickness}<0\f$ . The example below shows how to retrieve
    .   connected components from the binary image and label them: :
    .   @include snippets/imgproc_drawContours.cpp
    .   
    .   @param image Destination image.
    .   @param contours All the input contours. Each contour is stored as a point vector.
    .   @param contourIdx Parameter indicating a contour to draw. If it is negative, all the contours are drawn.
    .   @param color Color of the contours.
    .   @param thickness Thickness of lines the contours are drawn with. If it is negative (for example,
    .   thickness

In [56]:
import cv2
import numpy as np


#orb = cv2.ORB()
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

minpoint = []
mindes = []
def findNearestKeyPoint(x,y,kp,des):
    global minpointP, mindes
    mindist = 100000000000
    minpoint = kp[0]
    mindes = des[0]
    index = 0
    for point in kp:
        dist = (point.pt[0]-x)**2+(point.pt[1]-y)**2
        if dist < mindist:
            mindist = dist
            minpoint = point
            mindes = des[index]
            #print des[index]
        index += 1


def service_mouse(event,x,y,flags,param):
    global kp, des
    if event == cv2.EVENT_LBUTTONDOWN:
        findNearestKeyPoint(x,y,kp,des)


cap = cv2.VideoCapture(0)

cv2.namedWindow('image')
cv2.setMouseCallback('image',service_mouse)

while(1):

    _, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp = orb.detect(gray,None)
    kp, des = orb.compute(gray, kp)
    img2 = cv2.drawKeypoints(frame,kp, frame, color=(0,255,0), flags=0)
    if minpoint:
        cv2.circle(img2, (int(minpoint.pt[0]),int(minpoint.pt[1])) , 4, [0,0,255],-1 )

        matches = bf.match(np.array([mindes]),des)
        # suggestions: get list of best, pick closest to knwon position, update descriptors

        matches = sorted(matches, key = lambda x:x.distance)
        newguy = kp[matches[0].trainIdx].pt
        cv2.circle(img2, (int(newguy[0]),int(newguy[1])) , 10, [0,0,255],-1 )


    cv2.imshow('image',cv2.pyrDown(img2))

    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

In [9]:
import cv2
import numpy as np


class MyCam():
    def __init__(self, frameSize=(480, 640), focus=600, avgPointPos=np.array([0, 0, 3]), sigma=.5, pointNum=300):
        self.pointCloud = sigma * np.random.randn(pointNum, 3)
        self.pointCloud = map(lambda pnt: pnt + avgPointPos, self.pointCloud)
        self.t = np.zeros(3)
        self.R = np.identity(3)
        self.frameSize = frameSize
        self.focus = focus

    def read(self):
        pnts = np.array(self.projectPoints())
        frame = np.zeros(self.frameSize + (3,))

        for pnt in pnts.astype(int):
            if pnt[0] > 0 and pnt[1] > 1 and pnt[0] < self.frameSize[0] and pnt[1] < self.frameSize[1]:
                cv2.circle(frame, tuple(pnt), 5, [0, 0, 255], -1)
        return frame

    def addVecToPoint(self, points, vec):
        return map(lambda pnt: pnt + vec, points)

    def transformPoints(self):
        rotated = np.dot(self.pointCloud, self.R.T)
        translated = self.addVecToPoint(rotated, self.t)
        return translated

    def projectPoints(self):
        transformed = self.transformPoints()
        inFrontofCameraPoints = filter(lambda pnt: pnt[2] > 0, transformed)
        return map(lambda pnt: self.focus * pnt[:2]/pnt[2] + np.array(self.frameSize)/2, inFrontofCameraPoints)


cam = MyCam()
'''
frame = cam.read()
cv2.imshow('frame',frame)
k = cv2.waitKey(0)
'''

angle = .1

rotateZ = np.array([[np.cos(angle), np.sin(angle), 0],
                    [-np.sin(angle), np.cos(angle), 0],
                    [0, 0, 1]])
while(1):
    frame = cam.read()
    cv2.imshow('frame', frame)
    cam.R = np.dot(rotate, cam.R)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

rotateX = np.array([[np.cos(angle), np.sin(angle), 0],
                    [-np.sin(angle), np.cos(angle), 0],
                    [0, 0, 1]])

cv2.destroyAllWindows()


TypeError: unsupported operand type(s) for *: 'map' and 'float'